# Homework 1

## Imports

In [1]:
import nbtools
nbtools.setup_nb()

In [2]:
from astropy import units, constants
import numpy
import plotly

# Homework code
from gr2 import tov

# Problem C

Common scaling constant $\rho_*$

In [3]:
rho_star = constants.m_n * constants.c ** 2 / ((1.0 * units.fm).to(units.m) ** 3)
rho_star / constants.M_sun

<Quantity 75706.2105281 1 / (m s2)>

## Problem C.1

### Establish EoS

In [4]:
gamma = 2.0
alpha = 0.5

n = 1 / (gamma - 1)
K_bar = 1.0 # used for simulation
K = alpha / (rho_star ** (gamma - 1)) # used for scaling results

### Compute TOV solutions for various rest densities

Set densities

In [5]:
mid_cent_density = 0.5 * rho_star
mid_cent_density_bar = (K ** n) * mid_cent_density

In [6]:
mid_cent_density_bar

<Quantity 0.25>

In [7]:
densities_rest = numpy.arange(0.1, 1.5, 0.1)
pressures_cent = tov.polytropic_pressure(densities_rest, index=n, gas_const=K_bar)
densities_cent = tov.polytropic_density(pressure=pressures_cent, index=n, gas_constant=K_bar)

d_rad = 0.001
rad_max = 2.0
solver = tov.integrate_manual # method used by Prof. Radice in NR

Compute solutions

In [8]:
seq = tov.SolutionSequence.from_densities(densities_rest=densities_rest, d_rad=d_rad, 
                                          poly_index=n, poly_gas_const=K_bar, 
                                          rad_max=rad_max, solver=solver)

In [10]:
_pressures_cent = tov.polytropic_pressure(seq.densities_rest, index=n, gas_const=K_bar)
_densities_cent = tov.polytropic_density(pressure=pressures_cent, index=n, gas_constant=K_bar)
_densities_cent

array([0.11, 0.24, 0.39, 0.56, 0.75, 0.96, 1.19, 1.44, 1.71, 2.  , 2.31,
       2.64, 2.99, 3.36])

In [12]:
(_densities_cent / (n * K_bar)) ** (n / (2 * n + 1))

array([0.47914199, 0.6214465 , 0.73061436, 0.82425706, 0.9085603 ,
       0.98648483, 1.0596985 , 1.12924323, 1.19581878, 1.25992105,
       1.32191641, 1.38208465, 1.44064529, 1.49777448])

In [13]:
seq.densities_rest

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4])